In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.master('local[*]').appName('Hamshahri_1').getOrCreate()

text_df_with_filename = spark.read.text("./Hamshahri/Hamshahri").withColumn("filename", substring_index(input_file_name(), "/", -1))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/15 15:48:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/15 15:48:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import re

def tokenize_2gram_text(text):
    cleaned_text = re.sub(r'[^\w\s\u0600-\u06FF]', ' ', text)
    cleaned_text = re.sub(r'\؟', ' ', cleaned_text)
    words = cleaned_text.split()
    if 'title' in words:
        words.remove('title')
    if 'text' in words:
        words.remove('text')
    ngrams = []
    for i in range(len(words) - 2):
        ngram = words[i] + " " + words[i + 1] + " " + words[i + 2]
        ngrams.append(ngram)
    return ngrams

In [4]:
tokenize_2gram_text_udf = udf(tokenize_2gram_text, ArrayType(StringType()))

In [5]:

specific_item = text_df_with_filename.filter(col("filename") == 'HAM2-750403-045.txt').select(col("value")).first()

if specific_item is not None:
    text_of_specific_item = specific_item['value']
    print('specific_item: ', text_of_specific_item)
    a = tokenize_2gram_text(text_of_specific_item)
    print('Token: ', a)

specific_item:  {"title":"فرهنگ سياسي در جامعه ديني در گفت و گوبا حجت الاسلام والمسلمين دكتر سيد محمدخاتمي- قسمت آخر مولفه هاي فرهنگ سياسي معاصر","text":"بدينسان در فضاي عيني و ذهني زندگي مسلمانان كه به طور عمده تحت استيلاي تغلب، در واقعيت سياسي از يكسو و چيرگي تشرع و تصوف در باوروذهن مسلمانان از سوي ديگر بود، نوزاد فلسفه خيلي زود دچار علت وآفت شد; يعني بلافاصله پس از فارابي كه موسس فلسفه اسلامي است وفلسفه مدني بخش قابل توجهي از نظام فلسفي او است، انديشيدن درباره سياست، مورد غفلت يا تغافل پسينيان قرار گرفت و خردورزي، دامن كشان از ساحتي كه در آن سرنوشت اين جهاني آدمي رقم مي خورد به فضاي بيكران مابعدالطبيعه رانده شد و در اين بخش به فرازهاي بلندي هم رسيد. فلسفه اسلامي كه با فارابي آغاز شد منهاي بخش مدني آن، در مكتب والاي حكمت متعاليه، با تركيبي بديع ميان شاخه هاي مشاء، اشراق، كلام و نيزعرفان در واپسين بازدمهاي تمدن مسلمانان، به تعبيري به تماميت رسيدو پس از صدرالمتالهين، افتخار بزرگ دست اندركاران فلسفه در حوزه هاي معرفتهاي اسلامي نه عرضه انديشه و دبستان فلسفه اي نو، كه فهم و شرح فلسفه صدرا

In [6]:
tokenized_df = text_df_with_filename.withColumn("word", explode(tokenize_2gram_text_udf(col("value"))))


In [7]:
tokenized_df.show()

+--------------------+-------------------+--------------------+
|               value|           filename|                word|
+--------------------+-------------------+--------------------+
|{"title":"نام نوي...|HAM2-750405-016.txt|        نام نويسي در|
|{"title":"نام نوي...|HAM2-750405-016.txt|       نويسي در كدام|
|{"title":"نام نوي...|HAM2-750405-016.txt|       در كدام مدرسه|
|{"title":"نام نوي...|HAM2-750405-016.txt|     كدام مدرسه زياد|
|{"title":"نام نوي...|HAM2-750405-016.txt|      مدرسه زياد سخت|
|{"title":"نام نوي...|HAM2-750405-016.txt|     زياد سخت نگيريم|
|{"title":"نام نوي...|HAM2-750405-016.txt|      سخت نگيريم صبح|
|{"title":"نام نوي...|HAM2-750405-016.txt|      نگيريم صبح روز|
|{"title":"نام نوي...|HAM2-750405-016.txt|     صبح روز پنجشنبه|
|{"title":"نام نوي...|HAM2-750405-016.txt|      روز پنجشنبه 30|
|{"title":"نام نوي...|HAM2-750405-016.txt|    پنجشنبه 30 خرداد|
|{"title":"نام نوي...|HAM2-750405-016.txt|        30 خرداد ماه|
|{"title":"نام نوي...|HAM2-750405-016.tx

In [8]:
result_df = tokenized_df.groupBy("word", "filename").agg(
    count("word").cast('int').alias("count")
)

grouped_df = result_df.groupBy('word').agg(collect_list(struct('filename', 'count').alias('Name Repeat Count')))

grouped_df.show(truncate=False)

+--------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|word                            |collect_list(struct(filename, count) AS `Name Repeat Count`)                                                                      |
+--------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|000 180 ليتر                    |[{HAM2-750409-007.txt, 1}]                                                                                                        |
|048 19855ونزوئلا260 19837اندونزي|[{HAM2-750404-006.txt, 1}]                                                                                                        |
|1 بچه هربار،                    |[{HAM2-750404-085.txt, 1}]                                                                                                        |
|1 ت